In [1]:
import sys
sys.path.append('/SEE-MTDA/see')
from sc.mesher.mesher import Mesher
import numpy as np
import os
import glob
from pathlib import Path
from sc.datasets.shared_utils import *
import open3d as o3d
from easydict import EasyDict
import yaml

def cfg_from_yaml_file(cfg_file):
    cfg = EasyDict()
    print("\n----- Cfgs -----")
    with open(cfg_file, 'r') as f:
        yaml_config = yaml.safe_load(f)
        print(yaml.dump(yaml_config))

    cfg.update(EasyDict(yaml_config))

    return cfg

cfg_file = '/SEE-MTDA/see/sc/cfgs/KIT-DM-ORH005.yaml'
cfg = cfg_from_yaml_file(cfg_file)
mesher = Mesher(cfg=cfg, cfg_path=cfg_file)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.

----- Cfgs -----
DATASET:
  CAMERA_CHANNELS:
  - image_2
  CLASSES:
  - Car
  DET2D_MASK: true
  DET2D_MODEL: htc
  NAME: kitti
  POINT_CLOUD_RANGE:
  - -70
  - -70
  - -2
  - 70
  - 70
  - 4
  SHRINK_MASK_PERCENTAGE: 2
DATA_PATH: /SEE-MTDA/data/kitti
MESHER:
  CLUSTERING:
    EPS_SCALING: 5
    NAME: db_scan
  MESH_ALGORITHM:
    LOWER_RADIUS: 0.01
    NAME: ball_pivoting
    UPPER_RADIUS: 1.155
  MIN_LIDAR_PTS_TO_MESH: 30
  NAME: det_mesh
  REPLACE_OBJECT_WITH_MESH:
    POINT_DISTANCE_THRESH: 0.1
  SAMPLING:
    NAME: vres_ring_based_sampling
    OPTIMAL_RING_HEIGHT: 0.05
    TYPE: poisson
  VRES: 0.4

Loading masks...
loading annotations into memory...
Done (t=0.96s)
creating index...
index created!
KittiObjects initialised!

Mesher Initialised!


# Export points in GT box

In [2]:
mesher.min_lidar_pts_to_mesh = 5

for sample_idx in range(mesher.data_obj.__len__()):
    pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx)

    for object_id in range(len(pcd_gtboxes['gt_boxes'])):    

        object_pcd = pcd_gtboxes['pcd'].crop(pcd_gtboxes['gt_boxes'][object_id])

        if len(object_pcd.points) < mesher.min_lidar_pts_to_mesh:
            continue

        box_pts = np.asarray(pcd_gtboxes['gt_boxes'][object_id].get_box_points())

        if mesher.data_obj.dataset_name == 'nuscenes':
            fname = f'nsweeps-{mesher.data_obj.nsweeps}_frame-{sample_idx}_object-{object_id}_car_pts-{len(object_pcd.points)}'
        else:
            fname = f'frame-{sample_idx}_object-{object_id}_car_pts-{len(object_pcd.points)}'

        car_dir = Path(f'{mesher.data_obj.root_dir}/exported/cars')
        car_dir.mkdir(parents=True, exist_ok=True)
        o3d.io.write_point_cloud(str(car_dir / f'{fname}.pcd'), object_pcd)

        box_dir = Path(f'{mesher.data_obj.root_dir}/exported/bboxes')    
        box_dir.mkdir(parents=True, exist_ok=True)
        np.savetxt(str(box_dir / f'{fname}.txt'), box_pts, delimiter=' ')

        print(f"frame-{sample_idx}: Exported {object_id}/{len(pcd_gtboxes['gt_boxes'])} | car_pts-{len(object_pcd.points)}")


frame-1: Exported 0/1 | car_pts-9
frame-2: Exported 0/1 | car_pts-53


KeyboardInterrupt: 

In [115]:
sfd = []
if not sfd or max(sfd) == 0:
    print('yes')

yes


# Export points in mask/bbox

In [ ]:
export_tag = 'mask'
if export_tag == 'bbox':
    use_bbox = True


for sample_idx in range(mesher.data_obj.__len__()):
    
    i_clouds = mesher.data_obj.get_mask_instance_clouds(sample_idx, append_labels=False, use_bbox=True)
    pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx, add_ground_lift=False)
    
    for object_id, i_cloud in enumerate(i_clouds):
        mask_pcd = convert_to_o3dpcd(i_clouds[object_id])
        cropped_pcds, num_pts_in_gt_for_mask = [], []
        for i in range(len(pcd_gtboxes['gt_boxes'])):    
            cropped = mask_pcd.crop(pcd_gtboxes['gt_boxes'][i])
            num_pts_in_gt_for_mask.append(len(cropped.points))
            cropped_pcds.append(cropped)

        if not num_pts_in_gt_for_mask or max(num_pts_in_gt_for_mask) == 0:
            labelled_pts = np.hstack([i_cloud, np.ones((len(i_cloud),0))])
            box_pts = np.zeros((8,3))
            num_points_in_gt = 0
        else:
            gtbox_idx = np.argmax(np.array(num_pts_in_gt_for_mask)) 
            cropped_np = np.array(cropped_pcds[gtbox_idx].points)
            obj_pts_labelled = np.hstack([cropped_np, np.ones((len(cropped_np),1))])

            dist = mask_pcd.compute_point_cloud_distance(cropped_pcds[gtbox_idx])
            cropped_inds = np.where(np.asarray(dist) < 0.01)[0]
            mask_without_obj = np.asarray(mask_pcd.select_by_index(cropped_inds, invert=True).points)
            mask_pts_labelled = np.hstack([mask_without_obj, np.zeros((len(mask_without_obj),1))])\

            labelled_pts = np.vstack([mask_pts_labelled, obj_pts_labelled])
            box_pts = np.asarray(pcd_gtboxes['gt_boxes'][gtbox_idx].get_box_points())
            num_points_in_gt = len(cropped_np)
            
        if mesher.data_obj.dataset_name == 'nuscenes':
            fname = f'nsweeps-{mesher.data_obj.nsweeps}_frame-{sample_idx}_object-{object_id}_total_pts-{len(labelled_pts)}'
        else:
            fname = f'frame-{sample_idx}_object-{object_id}_total_pts-{len(labelled_pts)}'

        car_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data/points_in_{export_tag}/cars')
        car_dir.mkdir(parents=True, exist_ok=True)
        o3d.io.write_point_cloud(str(car_dir / f'{fname}.pcd'), object_pcd)

        box_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data/points_in_{export_tag}/gt_bboxes')    
        box_dir.mkdir(parents=True, exist_ok=True)
        np.savetxt(str(box_dir / f'{fname}.txt'), box_pts, delimiter=' ')

        print(f"frame-{sample_idx}: Exported {object_id+1}/{len(i_clouds)} | total pts-{len(labelled_pts)}, gt pts - {num_points_in_gt}")


frame-1: Exported 1/1 | total pts-10, gt pts - 9
frame-2: Exported 1/1 | total pts-110, gt pts - 67
frame-3: Exported 1/4 | total pts-408, gt pts - 0
frame-3: Exported 2/4 | total pts-952, gt pts - 674
frame-3: Exported 3/4 | total pts-2, gt pts - 0
frame-3: Exported 4/4 | total pts-15, gt pts - 0
frame-4: Exported 1/3 | total pts-120, gt pts - 68
frame-4: Exported 2/3 | total pts-33, gt pts - 25
frame-4: Exported 3/3 | total pts-3, gt pts - 0
frame-6: Exported 1/5 | total pts-666, gt pts - 321
frame-6: Exported 2/5 | total pts-138, gt pts - 65
frame-6: Exported 3/5 | total pts-98, gt pts - 26
frame-6: Exported 4/5 | total pts-10, gt pts - 0
frame-6: Exported 5/5 | total pts-16, gt pts - 0
frame-7: Exported 1/3 | total pts-213, gt pts - 182
frame-7: Exported 2/3 | total pts-25, gt pts - 20
frame-7: Exported 3/3 | total pts-7, gt pts - 5
frame-8: Exported 1/10 | total pts-2959, gt pts - 1371
frame-8: Exported 2/10 | total pts-1579, gt pts - 860
frame-8: Exported 3/10 | total pts-3099, g

frame-39: Exported 2/6 | total pts-613, gt pts - 240
frame-39: Exported 3/6 | total pts-115, gt pts - 68
frame-39: Exported 4/6 | total pts-122, gt pts - 81
frame-39: Exported 5/6 | total pts-18, gt pts - 16
frame-39: Exported 6/6 | total pts-33, gt pts - 21
frame-40: Exported 1/2 | total pts-85, gt pts - 60
frame-40: Exported 2/2 | total pts-17, gt pts - 8
frame-41: Exported 1/5 | total pts-93, gt pts - 56
frame-41: Exported 2/5 | total pts-27, gt pts - 0
frame-41: Exported 3/5 | total pts-14, gt pts - 0
frame-41: Exported 4/5 | total pts-1, gt pts - 0
frame-41: Exported 5/5 | total pts-8, gt pts - 0
frame-42: Exported 1/1 | total pts-25, gt pts - 22
frame-44: Exported 1/2 | total pts-2440, gt pts - 1642
frame-44: Exported 2/2 | total pts-840, gt pts - 559
frame-45: Exported 1/7 | total pts-1064, gt pts - 615
frame-45: Exported 2/7 | total pts-345, gt pts - 173
frame-45: Exported 3/7 | total pts-113, gt pts - 62
frame-45: Exported 4/7 | total pts-102, gt pts - 30
frame-45: Exported 5/

frame-78: Exported 6/7 | total pts-4, gt pts - 2
frame-78: Exported 7/7 | total pts-2, gt pts - 0
frame-79: Exported 1/14 | total pts-1574, gt pts - 650
frame-79: Exported 2/14 | total pts-1958, gt pts - 938
frame-79: Exported 3/14 | total pts-45, gt pts - 21
frame-79: Exported 4/14 | total pts-7, gt pts - 0
frame-79: Exported 5/14 | total pts-36, gt pts - 21
frame-79: Exported 6/14 | total pts-21, gt pts - 0
frame-79: Exported 7/14 | total pts-6, gt pts - 0
frame-79: Exported 8/14 | total pts-17, gt pts - 2
frame-79: Exported 9/14 | total pts-12, gt pts - 0
frame-79: Exported 10/14 | total pts-3, gt pts - 0
frame-79: Exported 11/14 | total pts-6, gt pts - 0
frame-79: Exported 12/14 | total pts-4, gt pts - 0
frame-79: Exported 13/14 | total pts-10, gt pts - 0
frame-79: Exported 14/14 | total pts-4, gt pts - 0
frame-80: Exported 1/6 | total pts-268, gt pts - 221
frame-80: Exported 2/6 | total pts-101, gt pts - 93
frame-80: Exported 3/6 | total pts-13, gt pts - 11
frame-80: Exported 4/6 

frame-110: Exported 1/13 | total pts-1813, gt pts - 765
frame-110: Exported 2/13 | total pts-640, gt pts - 432
frame-110: Exported 3/13 | total pts-328, gt pts - 179
frame-110: Exported 4/13 | total pts-499, gt pts - 353
frame-110: Exported 5/13 | total pts-143, gt pts - 114
frame-110: Exported 6/13 | total pts-75, gt pts - 0
frame-110: Exported 7/13 | total pts-111, gt pts - 72
frame-110: Exported 8/13 | total pts-33, gt pts - 17
frame-110: Exported 9/13 | total pts-42, gt pts - 30
frame-110: Exported 10/13 | total pts-45, gt pts - 40
frame-110: Exported 11/13 | total pts-65, gt pts - 23
frame-110: Exported 12/13 | total pts-15, gt pts - 3
frame-110: Exported 13/13 | total pts-1, gt pts - 0
frame-111: Exported 1/9 | total pts-1246, gt pts - 473
frame-111: Exported 2/9 | total pts-182, gt pts - 70
frame-111: Exported 3/9 | total pts-360, gt pts - 222
frame-111: Exported 4/9 | total pts-211, gt pts - 136
frame-111: Exported 5/9 | total pts-212, gt pts - 171
frame-111: Exported 6/9 | tot

frame-132: Exported 1/2 | total pts-126, gt pts - 102
frame-132: Exported 2/2 | total pts-2, gt pts - 2
frame-133: Exported 1/5 | total pts-86, gt pts - 55
frame-133: Exported 2/5 | total pts-47, gt pts - 37
frame-133: Exported 3/5 | total pts-5, gt pts - 0
frame-133: Exported 4/5 | total pts-15, gt pts - 12
frame-133: Exported 5/5 | total pts-9, gt pts - 6
frame-134: Exported 1/4 | total pts-1287, gt pts - 569
frame-134: Exported 2/4 | total pts-124, gt pts - 11
frame-134: Exported 3/4 | total pts-129, gt pts - 3
frame-134: Exported 4/4 | total pts-20, gt pts - 0
frame-135: Exported 1/12 | total pts-205, gt pts - 0
frame-135: Exported 2/12 | total pts-391, gt pts - 239
frame-135: Exported 3/12 | total pts-133, gt pts - 98
frame-135: Exported 4/12 | total pts-259, gt pts - 134
frame-135: Exported 5/12 | total pts-167, gt pts - 101
frame-135: Exported 6/12 | total pts-63, gt pts - 31
frame-135: Exported 7/12 | total pts-69, gt pts - 45
frame-135: Exported 8/12 | total pts-66, gt pts - 0

...

frame-5265: Exported 1/5 | total pts-2395, gt pts - 1418
frame-5265: Exported 2/5 | total pts-930, gt pts - 302
frame-5265: Exported 3/5 | total pts-404, gt pts - 216
frame-5265: Exported 4/5 | total pts-171, gt pts - 0
frame-5265: Exported 5/5 | total pts-60, gt pts - 45
frame-5266: Exported 1/12 | total pts-766, gt pts - 414
frame-5266: Exported 2/12 | total pts-456, gt pts - 272
frame-5266: Exported 3/12 | total pts-551, gt pts - 321
frame-5266: Exported 4/12 | total pts-248, gt pts - 129
frame-5266: Exported 5/12 | total pts-58, gt pts - 0
frame-5266: Exported 6/12 | total pts-44, gt pts - 0
frame-5266: Exported 7/12 | total pts-85, gt pts - 12
frame-5266: Exported 8/12 | total pts-7, gt pts - 0
frame-5266: Exported 9/12 | total pts-22, gt pts - 0
frame-5266: Exported 10/12 | total pts-19, gt pts - 0
frame-5266: Exported 11/12 | total pts-28, gt pts - 3
frame-5266: Exported 12/12 | total pts-2, gt pts - 0
frame-5267: Exported 1/2 | total pts-200, gt pts - 120
frame-5267: Exported 2

frame-5295: Exported 1/9 | total pts-343, gt pts - 207
frame-5295: Exported 2/9 | total pts-127, gt pts - 0
frame-5295: Exported 3/9 | total pts-118, gt pts - 53
frame-5295: Exported 4/9 | total pts-44, gt pts - 38
frame-5295: Exported 5/9 | total pts-3, gt pts - 0
frame-5295: Exported 6/9 | total pts-23, gt pts - 15
frame-5295: Exported 7/9 | total pts-15, gt pts - 0
frame-5295: Exported 8/9 | total pts-27, gt pts - 1
frame-5295: Exported 9/9 | total pts-17, gt pts - 7
frame-5296: Exported 1/2 | total pts-185, gt pts - 117
frame-5296: Exported 2/2 | total pts-29, gt pts - 0
frame-5297: Exported 1/4 | total pts-41, gt pts - 12
frame-5297: Exported 2/4 | total pts-11, gt pts - 9
frame-5297: Exported 3/4 | total pts-23, gt pts - 3
frame-5297: Exported 4/4 | total pts-2, gt pts - 0
frame-5298: Exported 1/5 | total pts-229, gt pts - 88
frame-5298: Exported 2/5 | total pts-229, gt pts - 151
frame-5298: Exported 3/5 | total pts-85, gt pts - 81
frame-5298: Exported 4/5 | total pts-10, gt pts 

frame-5331: Exported 1/1 | total pts-117, gt pts - 99
frame-5332: Exported 1/13 | total pts-1576, gt pts - 1008
frame-5332: Exported 2/13 | total pts-413, gt pts - 162
frame-5332: Exported 3/13 | total pts-59, gt pts - 0
frame-5332: Exported 4/13 | total pts-40, gt pts - 0
frame-5332: Exported 5/13 | total pts-45, gt pts - 0
frame-5332: Exported 6/13 | total pts-28, gt pts - 0
frame-5332: Exported 7/13 | total pts-74, gt pts - 2
frame-5332: Exported 8/13 | total pts-43, gt pts - 22
frame-5332: Exported 9/13 | total pts-35, gt pts - 0
frame-5332: Exported 10/13 | total pts-23, gt pts - 0
frame-5332: Exported 11/13 | total pts-29, gt pts - 11
frame-5332: Exported 12/13 | total pts-7, gt pts - 5
frame-5332: Exported 13/13 | total pts-18, gt pts - 0
frame-5333: Exported 1/6 | total pts-944, gt pts - 358
frame-5333: Exported 2/6 | total pts-233, gt pts - 156
frame-5333: Exported 3/6 | total pts-92, gt pts - 31
frame-5333: Exported 4/6 | total pts-146, gt pts - 95
frame-5333: Exported 5/6 | 

frame-5362: Exported 1/6 | total pts-1554, gt pts - 879
frame-5362: Exported 2/6 | total pts-580, gt pts - 248
frame-5362: Exported 3/6 | total pts-216, gt pts - 88
frame-5362: Exported 4/6 | total pts-120, gt pts - 68
frame-5362: Exported 5/6 | total pts-8, gt pts - 6
frame-5362: Exported 6/6 | total pts-45, gt pts - 24
frame-5363: Exported 1/1 | total pts-21, gt pts - 14
frame-5364: Exported 1/1 | total pts-235, gt pts - 219
frame-5365: Exported 1/5 | total pts-434, gt pts - 243
frame-5365: Exported 2/5 | total pts-148, gt pts - 45
frame-5365: Exported 3/5 | total pts-78, gt pts - 39
frame-5365: Exported 4/5 | total pts-44, gt pts - 37
frame-5365: Exported 5/5 | total pts-16, gt pts - 0
frame-5366: Exported 1/11 | total pts-2075, gt pts - 936
frame-5366: Exported 2/11 | total pts-1606, gt pts - 1191
frame-5366: Exported 3/11 | total pts-1092, gt pts - 629
frame-5366: Exported 4/11 | total pts-680, gt pts - 257
frame-5366: Exported 5/11 | total pts-292, gt pts - 199
frame-5366: Export

frame-5389: Exported 1/7 | total pts-1712, gt pts - 1040
frame-5389: Exported 2/7 | total pts-483, gt pts - 63
frame-5389: Exported 3/7 | total pts-49, gt pts - 20
frame-5389: Exported 4/7 | total pts-15, gt pts - 4
frame-5389: Exported 5/7 | total pts-36, gt pts - 0
frame-5389: Exported 6/7 | total pts-4, gt pts - 0
frame-5389: Exported 7/7 | total pts-4, gt pts - 0
frame-5391: Exported 1/3 | total pts-3195, gt pts - 1699
frame-5391: Exported 2/3 | total pts-114, gt pts - 0
frame-5391: Exported 3/3 | total pts-32, gt pts - 22
frame-5392: Exported 1/8 | total pts-1234, gt pts - 344
frame-5392: Exported 2/8 | total pts-2307, gt pts - 1383
frame-5392: Exported 3/8 | total pts-717, gt pts - 442
frame-5392: Exported 4/8 | total pts-508, gt pts - 231
frame-5392: Exported 5/8 | total pts-331, gt pts - 170
frame-5392: Exported 6/8 | total pts-110, gt pts - 41
frame-5392: Exported 7/8 | total pts-69, gt pts - 35
frame-5392: Exported 8/8 | total pts-12, gt pts - 5
frame-5393: Exported 1/2 | tot